In [2]:
# Dependencies

import pandas as pd
import requests
import json
import pprint
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from us import states

# Census & gmaps API Keys
from config import (api_key, gkey)
year = 2012
c = Census(api_key, year)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [3]:
datalst = []

In [4]:
for year in range(2012, 2018):
    c = Census(api_key, year) 

    data = c.acs5.get(('NAME', "B25077_001E","B25064_001E", "B15003_022E","B19013_001E"), 
                      {'for': 'zip code tabulation area:*'}) 

    cleandata = (pd.DataFrame(data) 
                   .rename(columns={"NAME": "Name", 
                                    "zip code tabulation area": "Zipcode", 
                                    "B25077_001E": "Median_Home_Value", 
                                    "B25064_001E": "Median_Rent", 
                                    "B15003_022E": "Bachelor_Degrees", 
                                    "B19013_001E": "Median_Income"}) 
                   .astype({'Zipcode':'int64'}) 
                   .query('(Median_Home_Value > 0) & (Median_Rent > 0) & (Median_Income > 0)')
                   .dropna()
                   .assign(Year = year)
                 )

    datalst.append(cleandata)


final_data = pd.concat(datalst, ignore_index = True)

In [42]:
testdf = final_data.groupby(['Name','Year'])


In [43]:
testdf.mean()

Median_Home_Value  Median_Rent  Bachelor_Degrees  \
Name        Year                                                     
ZCTA5 00601 2012           111000.0        370.0            1595.0   
            2013           104600.0        381.0            1630.0   
            2014           105400.0        367.0            1617.0   
            2015           104500.0        359.0            1720.0   
            2016            92000.0        375.0            1845.0   
...                             ...          ...               ...   
ZCTA5 99929 2013           157300.0        772.0             242.0   
            2014           168100.0        692.0             288.0   
            2015           172600.0        732.0             262.0   
            2016           185300.0        759.0             247.0   
            2017           195900.0        793.0             239.0   

                  Median_Income  Zipcode  
Name        Year                          
ZCTA5 00601 2012        13495.0      601  
            2013        12041.0      601  
            2014        10833.0      601  
            2015        10816.0      601  
            2016        11507.0      601  
...                         ...      ...  
ZCTA5 99929 2013        45649.0    99929  
            2014        48153.0    99929  
            2015        47941.0    99929  
            2016        52452.0    99929  
            2017        55742.0    99929  

[163780 rows x 5 columns]

In [51]:
testdf.loc('ZCTA5 99929')

AttributeError: Cannot access callable attribute 'loc' of 'DataFrameGroupBy' objects, try using the 'apply' method